# Imports

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import re

In [46]:
data = {
    'raw_message': [
        'hello',
        'hi', # for some reason this triggers really large values in things like 'bio' and 'cognitive mech'
        'hi!',
        'I am happy today', # first_person is always 0
        'i am good', # first_person is always 0
        '()()()()', # all symbols leads to the regex returning len(regex),
        ':-|', # another all symbol one
        'I think perhaps this is maybe fine', # hedge words
        'who what when where why',
        'I am good happy wonderful great excellent', # lots of positive words
        'under the sea and above the waves', # prepositions,
        'i can see how the family is upset because they feel the mother was disrespected but i can also understand the guy\'s feelings. why should he have to work as interpreter for his mother in law?', # TODO - weirdly, the outputs in the real thing differ from the outputs in the test here...
        'i was conflicted because i could understand his frustration however i feel he should have maybe discussed strategies with how to approach the mother in law with his wife first.'
    ]
}

def preprocess_text(text):
  	# For each individual message: preprocess to remove anything that is not an alphabet or number from the string
	return(re.sub(r"[^a-zA-Z0-9 ]+", '',text).lower())

chat_df = pd.DataFrame(data)

chat_df["message"] = chat_df["raw_message"].apply(preprocess_text)

In [47]:
chat_df

,raw_message,message
0,hello,hello
1,hi,hi
2,hi!,hi
3,I am happy today,i am happy today
4,i am good,i am good
5,()()()(),
6,:-|,
7,I think perhaps this is maybe fine,i think perhaps this is maybe fine
8,who what when where why,who what when where why
9,I am good happy wonderful great excellent,i am good happy wonderful great excellent


In [48]:
with open("../../features/lexicons_dict.pkl", "rb") as lexicons_pickle_file:
    lexicons_dict = pickle.load(lexicons_pickle_file)

In [49]:
test_df = pd.concat(
		# Finding the # of occurances of lexicons of each type for all the messages.
		[pd.DataFrame(chat_df["message"].apply(lambda chat: len(re.findall(regex, chat))))\
			  							.rename({"message": lexicon_type}, axis=1)\
			for lexicon_type, regex in lexicons_dict.items()], 
		axis=1
	)

In [50]:
test_df

,discrepancies,hear,home,conjunction,certainty,inclusive,bio,achievement,adverbs,anxiety,...,auxiliary_verbs,cognitive_mech,preposition,first_person_plural,percept,second_person,positive_words,first_person,nltk_english_stopwords,hedge_words
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,2,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,2,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,2,0,...,1,3,0,0,0,0,1,1,3,1
8,0,0,0,1,0,0,0,0,2,0,...,0,1,0,0,0,0,0,0,5,0
9,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,5,1,2,0


A few potential error cases below --- it seems we are returning more positives than necessary!

In [51]:
text = 'i can see how the family is upset because they feel the mother was disrespected but i can also understand the guy\'s feelings. why should he have to work as interpreter for his mother in law?'
regexs = lexicons_dict["cognitive_mech"].split('|')

regexs_to_test = []
for regex in regexs:
    if re.findall(regex, text) != []:
        regexs_to_test.append(regex)
        print(regex, re.findall(regex, text))

\bhow\b ['how']
\binterpret\S*\b ['interpreter']
\bbecause\b ['because']
\bunderstand\b ['understand']
\blaw\b ['law']
\bbut\b ['but']
\bfeel\b ['feel']
\bfeeling\S*\b ['feelings']
\bwhy\b ['why']
\bshould\b ['should']


kind (of)

In [52]:
# test_regex = '|'.join(lexicons_dict["cognitive_mech"].split('|')[50:100]) # how

test_regex = '|'.join(lexicons_dict["cognitive_mech"].split('|')[360:400])
re.findall(test_regex, text)

['because', 'understand']

In [53]:
test_regex

'\\bbecause\\b|\\bduti\\S*\\b|\\bkeep\\b|\\bpurpose\\S*\\b|\\buncertain\\S*\\b|\\bbecome\\b|\\bduty\\b|\\bkeeping\\S*\\b|\\bpuzzl\\S*\\b|\\bunclear\\S*\\b|\\bbecomes\\b|\\beach\\b|\\bkeeps\\b|\\bquer\\S*\\b|\\bundecided\\S*\\b|\\bbecoming\\b|\\beffect\\S*\\b|\\bkept\\b|\\bquestion\\S*\\b|\\bundeniab\\S*\\b|\\bbelief\\S*\\b|\\beither\\b|\\bkind of\\b|\\bquite\\b|\\bunderstand\\b|\\bbelieve\\b|\\belicit\\S*\\b|\\bkinda\\b|\\brandom\\S*\\b|\\bunderstandab\\S*\\b|\\bbelieved\\b|\\benclos\\S*\\b|\\bkindof\\b|\\brather\\b|\\bunderstanding\\S*\\b|\\bbelieves\\b|\\benlighten\\S*\\b|\\bknew\\b|\\brational\\S*\\b|\\bunderstands\\b'

In [54]:
regexs_to_test

['\\bhow\\b',
 '\\binterpret\\S*\\b',
 '\\bbecause\\b',
 '\\bunderstand\\b',
 '\\blaw\\b',
 '\\bbut\\b',
 '\\bfeel\\b',
 '\\bfeeling\\S*\\b',
 '\\bwhy\\b',
 '\\bshould\\b']

In [55]:
regex_to_test = '|'.join(regexs_to_test)
regex_to_test

'\\bhow\\b|\\binterpret\\S*\\b|\\bbecause\\b|\\bunderstand\\b|\\blaw\\b|\\bbut\\b|\\bfeel\\b|\\bfeeling\\S*\\b|\\bwhy\\b|\\bshould\\b'

In [56]:
re.findall(regex_to_test, text)
re.findall(lexicons_dict["cognitive_mech"], text)

['how',
 'because',
 'feel',
 'but',
 'understand',
 'feelings',
 'why',
 'should',
 'interpreter',
 'law']

In [57]:
regex_to_test

'\\bhow\\b|\\binterpret\\S*\\b|\\bbecause\\b|\\bunderstand\\b|\\blaw\\b|\\bbut\\b|\\bfeel\\b|\\bfeeling\\S*\\b|\\bwhy\\b|\\bshould\\b'

In [58]:
re.match(lexicons_dict["cognitive_mech"], text)
re.match(regex_to_test, text)

In [59]:
re.search(lexicons_dict["cognitive_mech"], text)

<re.Match object; span=(10, 13), match='how'>

In [60]:
for regex in regexs:
    if re.findall(regex, text) != []:

        print(regex, re.findall(regex, text))

\bhow\b ['how']
\binterpret\S*\b ['interpreter']
\bbecause\b ['because']
\bunderstand\b ['understand']
\blaw\b ['law']
\bbut\b ['but']
\bfeel\b ['feel']
\bfeeling\S*\b ['feelings']
\bwhy\b ['why']
\bshould\b ['should']


In [61]:
re.findall(
    '\\bkind (of)\\b', 
    'i can see how the family is upset because they feel the mother was disrespected but i can also understand the guy\'s feelings. why should he have to work as interpreter for his mother in law?'
)

[]

In [62]:
re.findall(lexicons_dict["cognitive_mech"], "i can see how the family is upset because they feel the mother was disrespected but i can also understand the guy\'s feelings. why should he have to work as interpreter for his mother in law?")

['how',
 'because',
 'feel',
 'but',
 'understand',
 'feelings',
 'why',
 'should',
 'interpreter',
 'law']

In [63]:
re.findall(lexicons_dict["preposition"], "i can see how the family is upset because they feel the mother was disrespected but i can also understand the guy\'s feelings. why should he have to work as interpreter for his mother in law?")

['to', 'as', 'for', 'in']